In [1]:
print('hello')

hello


In [29]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
MODEL="mistral:7b-instruct"


## ChatGPT API and ollama on local


In [30]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama

if MODEL.startswith("mistral"):
    model = Ollama(model=MODEL)




else :
    model = ChatOpenAI(
        model_name=MODEL,
        openai_api_key=OPENAI_API_KEY)



model.invoke("tell me a joke")
# %%


" Why don't scientists trust atoms?\n\nBecause they make up everything! (Everything is the punchline here, not everything in the universe)"

### basic Rag

load french code labour and split it as pages

In [32]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("./LEGITEXT000006072050.pdf")

pages = loader.load_and_split()

pages
#

from langchain.chains.question_answering import load_qa_chain


In [35]:
print(pages[0])


page_content='Code du travail
Partie législative
Chapitre préliminaire : Dialogue social.
Article L1
 
Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives
du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et
interprofessionnelle fait l'objet d'une concertation préalable avec les organisations syndicales de salariés et
d'employeurs représentatives au niveau national et interprofessionnel en vue de l'ouverture éventuelle d'une
telle négociation.
 
A cet effet, le Gouvernement leur communique un document d'orientation présentant des éléments de
diagnostic, les objectifs poursuivis et les principales options.
 
Lorsqu'elles font connaître leur intention d'engager une telle négociation, les organisations indiquent
également au Gouvernement le délai qu'elles estiment nécessaire pour conduire la négociation.
 
Le présent article n'est pas applicable en cas d'urgence. Lorsque le Gouve

# now define  an article
an article we be define in french  like this:
structure of the article:
1. theme
2. section
3. article
4. content

               { "theme": current_theme,
                "section": current_section,
                "article": article_text,
                "content":

#### extract the article and structure it like explained above


In [41]:
import re

section_pattern = re.compile(r'^Section \d+')
article_pattern = re.compile(r'^Article L\d+-\d+')
theme_pattern = re.compile(r'^Livre [IVXLCDM]+|Titre [IVXLCDM]+|Chapitre [IVXLCDM]+')



In [38]:
current_theme = None
current_section = None
current_article = None
all_structured_articles = []

for page in pages:
    lines = page.page_content.split("\n")
    for line in lines:
        # Detect a theme (e.g., "Livre", "Titre", "Chapitre")
        if theme_pattern.match(line):
            current_theme = line.strip()
            continue
        # Detect a section
        if section_pattern.match(line):
            current_section = line.strip()
            continue
        # Detect an article
        if article_pattern.match(line):
            current_article = line.strip()
            all_structured_articles.append({
                "theme": current_theme,
                "section": current_section,
                "article": current_article,
                "content": ""
            })
            continue
        # Add content to the current article
        if all_structured_articles:
            all_structured_articles[-1]["content"] += line + "\n"

### save the structured articles

In [44]:
import json

with open("structured_articles.json", "w", encoding="utf-8") as f:
    json.dump(all_structured_articles, f, ensure_ascii=False, indent=2)

    print("JSON file saved successfully.")


{'theme': "Titre Ier : Champ d'application et calcul des seuils d'effectifs", 'section': None, 'article': 'Article L1111-1', 'content': " \nLes dispositions du présent livre sont applicables aux employeurs de droit privé ainsi qu'à leurs salariés.\n \nElles sont également applicables au personnel des personnes publiques employé dans les conditions du\ndroit privé, sous réserve des dispositions particulières ayant le même objet résultant du statut qui régit ce\npersonnel.\n"}
JSON file saved successfully.


#### storage and indexing

In [49]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

texts = [article['content'] for article in all_structured_articles]
metadatas = [
    {
        "theme": article["theme"],
        "section": article["section"],
        "article": article["article"]
    }
    for article in all_structured_articles
]

#
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embeddings = HuggingFaceEmbeddings(model_name="dangvantuan/sentence-camembert-large")  # more accurate for french

#
db = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    persist_directory="french_labour_code_vectordb"
)
db.persist()

print("Base vectorielle ChromaDB créée et persistée dans 'french_labour_code_vectordb'")

/tmp/ipykernel_13434/533568030.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/INT/idrissou.f/PycharmProjects/RagProject/FrenchLabourCode/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Base vectorielle ChromaDB créée et persistée dans 'french_labour_code_vectordb'


/tmp/ipykernel_13434/533568030.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [50]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# load vector db
db = Chroma(
    persist_directory="french_labour_code_vectordb",
    embedding_function=embeddings
)

/tmp/ipykernel_13434/3726591181.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [51]:
query = "droit de grève"
results = db.similarity_search(query, k=3)  # k = nombre de résultats

for doc in results:
    print("Texte trouvé :")
    print(doc.page_content)
    print("Métadonnées :", doc.metadata)
    print("-" * 40)

Texte trouvé :
 
L'exercice du droit de grève ne peut justifier la rupture du contrat de travail, sauf faute lourde imputable au
salarié.
 
 
Son exercice ne peut donner lieu à aucune mesure discriminatoire telle que mentionnée à l'article L. 1132-2,
notamment en matière de rémunérations et d'avantages sociaux.
 
 
Tout licenciement prononcé en absence de faute lourde est nul de plein droit.
 

Métadonnées : {'section': 'Section 2 : Indemnisation du préjudice.', 'theme': 'Chapitre Ier : Dispositions générales.', 'article': 'Article L2511-1'}
----------------------------------------
Texte trouvé :
 
Les dispensateurs de formation de droit privé établissent, chaque année, un bilan, un compte de résultat et
une annexe dans des conditions déterminées par décret.
 
 

Métadonnées : {'section': 'Section 3 : Obligations comptables', 'theme': 'Chapitre II : Fonctionnement', 'article': 'Article L6352-6'}
----------------------------------------
Texte trouvé :
 
En Guadeloupe, en Guyane, en Mart

### Answering questions with rag chain

In [53]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

custom_prompt = PromptTemplate(
    template="""
Vous êtes un assistant juridique expert du Code du travail français.
Voici la question posée par l'utilisateur :
{question}

Voici le contexte extrait du Code du travail (articles pertinents) :
{context}

Consignes :
- Répondez uniquement en vous appuyant sur le contexte fourni.
- Citez explicitement les articles du Code du travail utilisés dans la réponse.
- Si la réponse n'est pas dans le contexte, dites-le clairement.

Réponse :
""",
    input_variables=["question", "context"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": custom_prompt}
)

In [54]:
user_question = "Je viens d'être embauchée, puis-je démissionner ?"
result = qa_chain({"query": user_question})
print(result["result"])

/tmp/ipykernel_13434/103748310.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": user_question})


 Oui, vous pouvez démissionner après avoir été embauchée. Cependant, il est important de respecter les conditions prévues par votre contrat de travail et par la loi sur le droit de se retirer du travail salarié.

En application de l'article L.3221-1 du Code du travail français, vous ne pouvez démissionner que dans un délai de deux mois suivant votre embauche.

Par ailleurs, en vertu de l'article L.4521-2, tout manquement à vos devoirs dans l'exercice de vos fonctions peut constituer une faute disciplinaire. Si vous démissionnez sans raison valide ou valable, votre employeur pourra éventuellement invoquer cette faute pour justifier d'un préjudice au titre du contrat de travail.

Enfin, en application de l'article L.132-5 du Code du travail, le contrat de travail est transmis au salarié, au plus tard, dans les deux jours ouvrables suivant l'embauche. Si ce n'est pas le cas, vous pourriez éventuellement invoquer ce manquement pour justifier un retard à votre démission.

Il est donc import

In [56]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Créer la mémoire
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Créer la chaîne RAG avec mémoire
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=db.as_retriever(),
    memory=memory
)

# Exemple d’utilisation
user_question = "Je viens d'être embauchée, puis-je démissionner ?"
result = qa_chain({"question": user_question})
print(result["answer"])

# Pour continuer la conversation, il suffit de rappeler la chaîne avec de nouvelles questions :
result2 = qa_chain({"question": "Et quelles sont les conditions pour un CDI ?"})
print(result2["answer"])

 Oui, vous pouvez démissionner à tout moment. Cependant, il est important de respecter les obligations légales et contractuelles liées à votre démission.
 Les conditions requises pour obtenir un contrat à durée indéterminée (CDI) ne sont pas explicitement mentionnées dans le contexte fourni. Cependant, en général, les conditions requises pour un CDI peuvent inclure que l'emploi soit permanent, qu'il y ait une convention collective ou un accord d'entreprise qui prévoie des contrats à durée indéterminée dans cette profession, et que le travail est réalisé de manière régulière et habituelle. Il est donc recommandé de consulter la législation locale sur les contrats de travail pour obtenir des informations précises.


In [57]:
import streamlit as st

st.title("Assistant Code du Travail")

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

user_input = st.text_input("Posez votre question :")

if st.button("Envoyer") and user_input:
    result = qa_chain({"question": user_input})
    st.session_state.chat_history.append(("user", user_input))
    st.session_state.chat_history.append(("ai", result["answer"]))

for speaker, message in st.session_state.chat_history:
    if speaker == "user":
        st.markdown(f"**Vous :** {message}")
    else:
        st.markdown(f"**Assistant :** {message}")

2025-07-25 13:25:12.309 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 13:25:12.370 
  command:

    streamlit run /home/INT/idrissou.f/PycharmProjects/RagProject/FrenchLabourCode/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-25 13:25:12.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 13:25:12.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 13:25:12.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 13:25:12.371 Session state does not function when running a script without `streamlit run`
2025-07-25 13:25:12.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 13: